In [1]:
"""
Deep Learning
Assignment 3
Previously in 2_fullyconnected.ipynb, you trained a logistic regression and a neural network model.
The goal of this assignment is to explore regularization techniques.
"""

# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [3]:
#First reload the data we generated in 1_notmnist.ipynb
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [4]:
"""
Reformat into a shape that's more adapted to the models we're going to train:
data as a flat matrix,
labels as float 1-hot encodings.
"""

image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [7]:
"""
PROBLEM 1
Introduce and tune L2 regularization for both logistic and neural network models. 
Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. 
In TensorFlow, you can compute the L2 loss for a tensor t using nn.l2_loss(t). 
The right amount of regularization should improve your validation / test accuracy
"""

# With logistic regression
train_subset = 10000
reg_term = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  unreg_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  l2_loss = reg_term * tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(unreg_loss + l2_loss)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [8]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 48.503315
Training accuracy: 5.7%
Validation accuracy: 7.6%
Loss at step 100: 11.620430
Training accuracy: 74.7%
Validation accuracy: 72.5%
Loss at step 200: 4.410895
Training accuracy: 79.2%
Validation accuracy: 76.7%
Loss at step 300: 1.948351
Training accuracy: 82.5%
Validation accuracy: 79.7%
Loss at step 400: 1.111343
Training accuracy: 83.9%
Validation accuracy: 81.2%
Loss at step 500: 0.823802
Training accuracy: 84.6%
Validation accuracy: 81.8%
Loss at step 600: 0.723802
Training accuracy: 84.7%
Validation accuracy: 81.9%
Loss at step 700: 0.688704
Training accuracy: 84.8%
Validation accuracy: 82.0%
Loss at step 800: 0.676299
Training accuracy: 84.8%
Validation accuracy: 82.0%
Test accuracy: 88.7%


In [10]:
#without L2 Reg  : Loss at step 800: 1.062301 | Training acc.: 79.6% | Validation acc.: 74.9% | Test acc.: 82.9%
#applying L2 Reg : Loss at step 800: 0.676299 | Training acc.: 84.8% | Validation acc.: 82.0% | Test acc.: 88.7%

In [44]:
## Problem 1 with one hidden layer network
# Parameters
num_steps = 30001 #training cycle | training_epochs
batch_size = 512
n_hidden_1 = 1024 # 1st layer number of features
n_input = image_size * image_size # data input (img shape: 28*28)
#num_labels = 10 #(0-9)

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

def oneHiddenLayerModel (x, weights, biases):
  # Hidden layer with RELU activation
  hiddenLayer = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
  hiddenLayer = tf.nn.relu(hiddenLayer)
  # Output layer with linear activation
  outputLayer = tf.add(tf.matmul(hiddenLayer, weights['out']), biases['out'])
  return outputLayer

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, n_input))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  reg_term = tf.placeholder(tf.float32) # regularization term (beta)
  learning_rate = tf.placeholder(tf.float32)
  
  # Variables : layers weight & bias
  weights = {
    'h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.truncated_normal([n_hidden_1, num_labels]))
  }
  biases = {
    'b1': tf.Variable(tf.zeros([n_hidden_1])),
    'out': tf.Variable(tf.zeros([num_labels]))
  }
 
  # Construct model
  pred = oneHiddenLayerModel(tf_train_dataset, weights, biases)

  # Loss (Cost)
  unreg_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=tf_train_labels))
  #l2_loss = reg_term * (tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(weights['out']))
  #loss = tf.reduce_mean(unreg_loss + l2_loss)
  loss = unreg_loss
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(pred)
  valid_prediction = tf.nn.softmax(oneHiddenLayerModel(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(oneHiddenLayerModel(tf_test_dataset, weights, biases))

In [45]:
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print ('Initialized')
  summary_data = np.zeros((num_steps, 3))

  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, reg_term : 1e-3, learning_rate : 0.05}
        
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

    if (step % 5000 == 0):
      valid_accuracy = accuracy(valid_prediction.eval(), valid_labels)
      summary_data[step] = [step, valid_accuracy, l]
      print ("Minibatch loss at step %d : %d - acc: %.1f%% | Validation acc: %.1f%%" % (step, l, accuracy(predictions, batch_labels),valid_accuracy))
  print ("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
  test_accuracy = accuracy(test_prediction.eval(), test_labels)
  print ("batch size = %d | num steps = %d | test acc. = %.1f%%" % (batch_size, num_steps, test_accuracy))

Initialized
Minibatch loss at step 0 : 291 - acc: 16.2% | Validation acc: 21.7%
Minibatch loss at step 5000 : 5 - acc: 86.5% | Validation acc: 83.5%
Minibatch loss at step 10000 : 3 - acc: 85.4% | Validation acc: 83.4%
Minibatch loss at step 15000 : 1 - acc: 87.5% | Validation acc: 83.4%
Minibatch loss at step 20000 : 1 - acc: 85.4% | Validation acc: 83.6%
Minibatch loss at step 25000 : 1 - acc: 85.2% | Validation acc: 84.5%
Minibatch loss at step 30000 : 1 - acc: 87.3% | Validation acc: 84.9%
Test accuracy: 91.7%


TypeError: a float is required

In [26]:
#batch size = 512 | num steps = 30001 | learning rate = 0.050 | regularization term = 0.0100000000 | test acc. = 91.3%
#batch size = 512 | num steps = 30001 | learning rate = 0.050 | regularization term = 0.0000000001 | test acc. = 91.4%
#batch size = 512 | num steps = 30001 | learning rate = 0.050 | no regularization term             | test acc. = 91.7%